In [1]:
pip install cvxpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np 
import pandas as pd 
import scipy.sparse as scs 
import scipy.linalg as scl 
import scipy.optimize as sco 


In [8]:
data_small1 = pd.read_csv('small1.csv')
data_small2 = pd.read_csv('small2.csv')
data_large1 = pd.read_csv('large1.csv')
data_large2 = pd.read_csv('large2.csv')

In [3]:
# constraints of soduku (reference: https://www.kaggle.com/gaz3ll3/sudoku-challenge-example-1)
def fixed_constraints(N=9):
    rowC = np.zeros(N)
    rowC[0] =1
    rowR = np.zeros(N)
    rowR[0] =1
    row = scl.toeplitz(rowC, rowR)
    ROW = np.kron(row, np.kron(np.ones((1,N)), np.eye(N)))
    
    colR = np.kron(np.ones((1,N)), rowC)
    col  = scl.toeplitz(rowC, colR)
    COL  = np.kron(col, np.eye(N))
    
    M = int(np.sqrt(N))
    boxC = np.zeros(M)
    boxC[0]=1
    boxR = np.kron(np.ones((1, M)), boxC) 
    box = scl.toeplitz(boxC, boxR)
    box = np.kron(np.eye(M), box)
    BOX = np.kron(box, np.block([np.eye(N), np.eye(N) ,np.eye(N)]))
    
    cell = np.eye(N**2)
    CELL = np.kron(cell, np.ones((1,N)))

    return scs.csr_matrix(np.block([[ROW],[COL],[BOX],[CELL]]))

In [4]:
fixed_constraints().toarray().shape

(324, 729)

In [5]:
def clue_constraint(input_quiz, N=9):
    m = np.reshape([int(c) for c in input_quiz], (N,N))
    r, c = np.where(m.T)
    v = np.array([m[c[d],r[d]] for d in range(len(r))])
    
    table = N * c + r
    table = np.block([[table],[v-1]])
    
    CLUE = scs.lil_matrix((len(table.T), N**3))
    for i in range(len(table.T)):
        CLUE[i,table[0,i]*N + table[1,i]] = 1

    CLUE = CLUE.tocsr() 
    
    return CLUE

In [11]:
import cvxpy as cp
import warnings
warnings.filterwarnings("ignore")
# the solver function
def solver(input):
    A0 = fixed_constraints()
    A1 = clue_constraint(input)

    A = scs.vstack((A0,A1))
    B = np.ones(A.toarray().shape[0])

    try:
        x = cp.Variable(A.toarray().shape[1])
        prob = cp.Problem(cp.Minimize(cp.norm(x, 1)),
                         [A.dot(x) == B, x>=0, x<=1])
        prob.solve(verbose=False)
        x = x.value
    except:
        x = np.zeros(A.toarray().shape[1])
        
    z = np.reshape(x, (81, 9))
    z = np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9))
    return ''.join([str(i) for i in z.flatten()])
    return z

In [6]:
def check(sol):
    solution=np.reshape([int(c) for c in sol], (9,9))
    for i in range(9):
        if [0,1,2,3,4,5,6,7,8]!=np.where(solution==(i+1))[0].tolist():
            return False
        k=i%3
        h=i//3
        block=solution[h*3:h*3+3,k*3:k*3+3]
        for j in range(9):
            [a,b]=np.shape(np.where(block==(j+1)))
            if b!=1:
                return False
        return True    

In [9]:
# Take a close look at one string in quizzes

quiz = data_small1["quizzes"][9]
print(quiz, '\ndata type is:', type(quiz))

# we can turn it into a numpy array by the following
np.reshape([int(c) for c in quiz], (9,9))

004029000000006703000000050100700036900000008380004002050000000603200000000310200 
data type is: <class 'str'>


array([[0, 0, 4, 0, 2, 9, 0, 0, 0],
       [0, 0, 0, 0, 0, 6, 7, 0, 3],
       [0, 0, 0, 0, 0, 0, 0, 5, 0],
       [1, 0, 0, 7, 0, 0, 0, 3, 6],
       [9, 0, 0, 0, 0, 0, 0, 0, 8],
       [3, 8, 0, 0, 0, 4, 0, 0, 2],
       [0, 5, 0, 0, 0, 0, 0, 0, 0],
       [6, 0, 3, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 3, 1, 0, 2, 0, 0]])

In [12]:
quiz = data_small1["quizzes"][0]
print(quiz)
print(solver(quiz))

080032001703080002500007030050001970600709008047200050020600009800090305300820010
489532761713486592562917834258341976631759248947268153125673489876194325394825617


# Result on Dataset A

In [13]:

n1 = len(data_small1['quizzes'])
cnt = 0
for i in range(n1):
    s = solver(data_small1['quizzes'][i])
    if s == data_small1['solutions'][i]:
        cnt += 1

n2 = len(data_small2['quizzes'])
for i in range(n2):
    s = solver(data_small2['quizzes'][i])
    if s == data_small2['solutions'][i]:
        cnt += 1
print('success rate is {}'.format(cnt/(n1+n2)))

success rate is 0.33719806763285026


In [16]:
data1=data_small1
n = len(data1['quizzes'])
k=0;
for i in range(n):
    s = solver(data1['quizzes'][i])
    if check(s):
        k+=1
    
print('success rate on small1 is', 100*k/n,'% ')

success rate on small1 is 100.0 % 


In [17]:
data2=data_small2
n = len(data2['quizzes'])
k=0;
for i in range(n):
    s = solver(data2['quizzes'][i])
    if check(s):
        k+=1
    
print('success rate on small2 is', 100*k/n,'% ')

success rate on small2 is 41.839762611275965 % 


# Result on Dataset B

In [14]:
random_seed = 42
np.random.seed(random_seed)
dataB1 = pd.read_csv('large1.csv')
n1 = len(dataB1['quizzes'])
cnt = 0
sample1 = np.random.choice(n1, 1000)
for i in sample1:
    s = solver(dataB1['quizzes'][i])
    if s == dataB1['solutions'][i]:
        cnt += 1
dataB2 = pd.read_csv('large2.csv')
n2 = len(dataB2['quizzes'])
sample2 = np.random.choice(n2, 1000)
for i in sample2:
    s = solver(dataB2['quizzes'][i])
    if s == dataB2['solutions'][i]:
        cnt += 1
print('success rate is {}'.format(cnt/2000))

success rate is 0.909


In [26]:
sample1.shape[0]

1000

In [29]:
data3=data_large1
n = sample1.shape[0]
k=0;
for i in sample1:
    s = solver(data3['quizzes'][i])
    if check(s):
        k+=1
    
print('success rate on small3 is', 100*k/n,'% ')

success rate on small3 is 89.9 % 


In [28]:
data4=data_large2
n = sample2.shape[0]
k=0;
for i in sample2:
    s = solver(data4['quizzes'][i])
    if check(s):
        k+=1
    
print('success rate on small4 is', 100*k/n,'% ')

success rate on small4 is 100.0 % 
